In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the Selenium WebDriver to simulate browser
driver = webdriver.Chrome()
driver.get("https://www.pricecharting.com/console/pokemon-japanese-scarlet-&-violet-151")

# Scroll to the bottom of the page multiple times to load all items
SCROLL_PAUSE_TIME = 2
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait to load the new content
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Detect whether page has ended
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break  # Break if no new content is loaded
    last_height = new_height

# Once the full content is loaded, parse the page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Scrape the data
products = soup.findAll("tr", {"id": lambda x: x and x.startswith("product-")})

In [10]:
# Extract and store data
data = []
for product in products:
    title_tag = product.find("td", class_="title")
    title = title_tag.text.strip() if title_tag else "No title found"
    
    if "#" in title:
        splitted = title.split("#")
        title = splitted[0].strip()
        card_number = splitted[-1].strip()
        
    else:
        card_number = 0
        
    if "[" in title:
        splitted = title.split("[")
        variant = splitted[-1][:-1].strip()
        title = splitted[0].strip()
    else:
        variant = "Normal"
    
    price_tag = product.find("span", class_="js-price").text.strip()
    price = float(price_tag[1:]) if price_tag else None
    
    data.append({"Title": title, "Card Number": card_number, "Price": price, "Variant": variant})

# Convert to DataFrame
SV = pd.DataFrame(data)
SV

,Title,Card Number,Price,Variant
0,Charizard EX,201,186.73,Normal
1,Mew EX,205,70.50,Normal
2,Pikachu,173,10.00,Normal
3,Mewtwo,183,8.75,Normal
4,Charmander,168,14.08,Normal
...,...,...,...,...
514,Cloyster,91,1.69,Normal
515,Gastly,92,1.65,Normal
516,Onix,95,1.60,Normal
517,Kingler,99,1.38,Normal
